In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm

In [35]:
# Example data
data = pd.DataFrame({
    'treatment': np.random.binomial(1, 0.5, 100),  # Binary treatment variable
    'covariate1': np.random.normal(0, 1, 100),
    'covariate2': np.random.normal(0, 1, 100),
    'covariate3': np.random.normal(0, 1, 100),
})
print(len(data))
data.head()

100


,treatment,covariate1,covariate2,covariate3
0,1,2.231389,0.673910,0.356429
1,1,-0.622559,1.848955,-1.404115
2,1,-0.550561,-0.485006,-0.311894
3,0,-0.919940,1.576381,0.607841
4,0,-0.786653,-0.179736,-1.458520


In [31]:
# Covariates
covariates = ['covariate1', 'covariate2', 'covariate3']

# Logistic regression
log_reg = LogisticRegression()
log_reg.fit(data[covariates], data['treatment'])

# Predict propensity scores
data['propensity_score'] = log_reg.predict_proba(data[covariates])[:, 1]

data.head()


,treatment,covariate1,covariate2,covariate3,propensity_score
0,0,0.351125,0.288528,-1.498305,0.568593
1,0,0.705708,-0.582951,-0.147879,0.466828
2,0,0.434737,-0.482779,0.181447,0.469472
3,1,0.368686,-0.409970,-1.648368,0.559636
4,1,0.245578,-0.236048,0.010052,0.493754


In [32]:
# Separate treated and control units
treated = data[data['treatment'] == 1]
control = data[data['treatment'] == 0]

# Initialize NearestNeighbors
nn = NearestNeighbors(n_neighbors=1)

# Fit on control propensity scores
nn.fit(control[['propensity_score']])

# Find nearest neighbors for treated units
distances, indices = nn.kneighbors(treated[['propensity_score']])

# Get matched control units and include the outcome column
matched_controls = control.iloc[indices.flatten()].copy()
matched_controls.index = treated.index  # Ensure indices match for concatenation

# Combine treated and matched control units
matched_data = pd.concat([treated, matched_controls], ignore_index=True)


In [33]:
def balance_table(data, covariates):
    balance = pd.DataFrame(index=covariates)
    for cov in covariates:
        mean_treated = data[data['treatment'] == 1][cov].mean()
        mean_control = data[data['treatment'] == 0][cov].mean()
        balance.loc[cov, 'Treated Mean'] = mean_treated
        balance.loc[cov, 'Control Mean'] = mean_control
        balance.loc[cov, 'Standardized Difference'] = (mean_treated - mean_control) / data[cov].std()
    return balance

# Before matching
print("Before Matching")
print(balance_table(data, covariates))

# After matching
print("After Matching")
print(balance_table(matched_data, covariates))


Before Matching
            Treated Mean  Control Mean  Standardized Difference
covariate1     -0.258578     -0.017865                -0.237590
covariate2     -0.156881     -0.252390                 0.081960
covariate3     -0.157828      0.047887                -0.193056
After Matching
            Treated Mean  Control Mean  Standardized Difference
covariate1     -0.258578     -0.448484                 0.184642
covariate2     -0.156881     -0.552626                 0.362191
covariate3     -0.157828     -0.034620                -0.125869


In [34]:
data['outcome'] = data['treatment'] + 0.5 * data['covariate1'] + np.random.normal(0, 1, 100)

# Check if 'outcome' column exists in 'matched_data' dataframe
if 'outcome' in matched_data.columns:
    # Outcome of matched data
    outcome_treated = matched_data[matched_data['treatment'] == 1]['outcome']
    outcome_control = matched_data[matched_data['treatment'] == 0]['outcome']

    # Average treatment effect on the treated (ATT)
    ATT = outcome_treated.mean() - outcome_control.mean()
    print(f"Average Treatment Effect on the Treated (ATT): {ATT}")
else:
    print("The 'outcome' column does not exist in the 'matched_data' dataframe.")


The 'outcome' column does not exist in the 'matched_data' dataframe.
